# Mongo to PostgreSQL for CWL Viewer

We only have two tables in the PostgreSQL schema, `workflow` and `queued_workflow`.

In [1]:
import pandas as pd
from uuid import uuid4
import json

In [2]:
%%time

# Load the JSON dump data

df = pd.read_json('2022-03-29T074829+0000.json')
df = df[['content']]
df = pd.json_normalize(df.content, max_level=0)
df.head()

CPU times: user 662 ms, sys: 43.3 ms, total: 706 ms
Wall time: 709 ms


,retrievedFrom,retrievedOn,lastCommit,label,inputs,outputs,steps,cwltoolVersion,visualisationDot,licenseLink,permalink,visualisationXdot,packed,visualisationPng,visualisationSvg,roBundle,@id,doc,dockerLink
0,{'repoUrl': 'https://github.com/stevencox/cwlt...,1648539658432,2f7df467b8e38c1ed0ea3c2280c25c49523da6cf,simple-workflow.cwl,{'echo_step_output_location': {'type': 'String'}},{'wf_output': {'type': 'File'}},"{'echo_step': {'run': 'echo.cwl', 'runType': '...",1.0.20180525185854,"digraph workflow {\n graph [\n bgcolor = ""...",https://raw.githubusercontent.com/stevencox/cw...,https://w3id.org/cwl/view/git/2f7df467b8e38c1e...,/graph/xdot/github.com/stevencox/cwltool/blob/...,False,/graph/png/github.com/stevencox/cwltool/blob/m...,/graph/svg/github.com/stevencox/cwltool/blob/m...,/robundle/github.com/stevencox/cwltool/blob/ma...,https://w3id.org/cwl/view/git/2f7df467b8e38c1e...,NaN,NaN
1,{'repoUrl': 'https://github.com/tmooney/cancer...,1648539591018,233f026ffce240071edda526391be0c03186fce8,Varscan Workflow,"{'p_value': {'type': 'Float (Optional)'}, 'ref...","{'filtered_vcf': {'type': 'File'}, 'unfiltered...","{'filter': {'label': 'fp_filter workflow', 'ru...",3.0.20201203173111,"digraph workflow {\n graph [\n bgcolor = ""...",https://raw.githubusercontent.com/tmooney/canc...,https://w3id.org/cwl/view/git/233f026ffce24007...,/graph/xdot/github.com/tmooney/cancer-genomics...,False,/graph/png/github.com/tmooney/cancer-genomics-...,/graph/svg/github.com/tmooney/cancer-genomics-...,/robundle/github.com/tmooney/cancer-genomics-w...,https://w3id.org/cwl/view/git/233f026ffce24007...,NaN,NaN
2,{'repoUrl': 'https://github.com/ebi-wp/webserv...,1648539494435,5df6b762980b15b0f6389149311b82bdd6dff37d,workflow-fetch-hmmscan.cwl,"{'database': {'type': 'String'}, 'method': {'t...",{'workflow_output': {'type': 'File'}},"{'hmmscan_step': {'run': 'hmmer3_hmmscan.cwl',...",3.1.20210922203925,"digraph workflow {\n graph [\n bgcolor = ""...",https://raw.githubusercontent.com/ebi-wp/webse...,https://w3id.org/cwl/view/git/5df6b762980b15b0...,/graph/xdot/github.com/ebi-wp/webservice-cwl/b...,False,/graph/png/github.com/ebi-wp/webservice-cwl/bl...,/graph/svg/github.com/ebi-wp/webservice-cwl/bl...,/robundle/github.com/ebi-wp/webservice-cwl/blo...,https://w3id.org/cwl/view/git/5df6b762980b15b0...,NaN,NaN
3,{'repoUrl': 'https://github.com/ProteinsWebTea...,1648539077414,56dafa4dab5892c5afa35713563dddc78ec5a00d,rRNA_selection.cwl,"{'indexed_sequences': {'type': 'File [FASTA]',...",{'matching_sequences': {'type': 'File [FASTA]'...,{'extract_coord_lines': {'run': 'extract_coord...,1.0.20180525185854,"digraph workflow {\n graph [\n bgcolor = ""...",https://www.apache.org/licenses/LICENSE-2.0,https://w3id.org/cwl/view/git/56dafa4dab5892c5...,/graph/xdot/github.com/ProteinsWebTeam/ebi-met...,False,/graph/png/github.com/ProteinsWebTeam/ebi-meta...,/graph/svg/github.com/ProteinsWebTeam/ebi-meta...,/robundle/github.com/ProteinsWebTeam/ebi-metag...,https://w3id.org/cwl/view/git/56dafa4dab5892c5...,NaN,NaN
4,{'repoUrl': 'https://github.com/rawgene/cwl.gi...,1648538897532,54adf7fcfbf48c9cc4cfcf384e04eede39b5e5f6,hisat2_htseq_dexseq.cwl,"{'annotation': {'type': 'File'}, 'htseq_prepar...","{'samtools_out': {'type': 'Directory'}, 'hisat...",{'htseq_prepare': {'run': '../tools/htseq_prep...,1.0.20180525185854,"digraph workflow {\n graph [\n bgcolor = ""...",https://raw.githubusercontent.com/rawgene/cwl/...,https://w3id.org/cwl/view/git/54adf7fcfbf48c9c...,/graph/xdot/github.com/rawgene/cwl/blob/master...,False,/graph/png/github.com/rawgene/cwl/blob/master/...,/graph/svg/github.com/rawgene/cwl/blob/master/...,/robundle/github.com/rawgene/cwl/blob/master/w...,https://w3id.org/cwl/view/git/54adf7fcfbf48c9c...,NaN,NaN


In [3]:
# Rename columns

def to_camel_case(snake_str):
    components = snake_str.split('_')
    # We capitalize the first letter of each component except the first one
    # with the 'title' method and join them together.
    return components[0] + ''.join(x.title() for x in components[1:])

# columns copied from CWL Viewer Java code (db migration)
# NOTE: in Java: ro_bundle_path, but Mongo had roBundle
workflow_columns=['cwltool_version', 'doc', 'docker_link', 'inputs', 'label', 'last_commit', 'license_link', 'outputs', 'retrieved_from', 'retrieved_on', 'steps', 'visualisation_dot']
workflow_columns={
    to_camel_case(k): k for k in workflow_columns
}
workflow_columns['roBundle'] = 'ro_bundle_path'

df = df.rename(columns=workflow_columns)
df.head()

,retrieved_from,retrieved_on,last_commit,label,inputs,outputs,steps,cwltool_version,visualisation_dot,license_link,permalink,visualisationXdot,packed,visualisationPng,visualisationSvg,ro_bundle_path,@id,doc,docker_link
0,{'repoUrl': 'https://github.com/stevencox/cwlt...,1648539658432,2f7df467b8e38c1ed0ea3c2280c25c49523da6cf,simple-workflow.cwl,{'echo_step_output_location': {'type': 'String'}},{'wf_output': {'type': 'File'}},"{'echo_step': {'run': 'echo.cwl', 'runType': '...",1.0.20180525185854,"digraph workflow {\n graph [\n bgcolor = ""...",https://raw.githubusercontent.com/stevencox/cw...,https://w3id.org/cwl/view/git/2f7df467b8e38c1e...,/graph/xdot/github.com/stevencox/cwltool/blob/...,False,/graph/png/github.com/stevencox/cwltool/blob/m...,/graph/svg/github.com/stevencox/cwltool/blob/m...,/robundle/github.com/stevencox/cwltool/blob/ma...,https://w3id.org/cwl/view/git/2f7df467b8e38c1e...,NaN,NaN
1,{'repoUrl': 'https://github.com/tmooney/cancer...,1648539591018,233f026ffce240071edda526391be0c03186fce8,Varscan Workflow,"{'p_value': {'type': 'Float (Optional)'}, 'ref...","{'filtered_vcf': {'type': 'File'}, 'unfiltered...","{'filter': {'label': 'fp_filter workflow', 'ru...",3.0.20201203173111,"digraph workflow {\n graph [\n bgcolor = ""...",https://raw.githubusercontent.com/tmooney/canc...,https://w3id.org/cwl/view/git/233f026ffce24007...,/graph/xdot/github.com/tmooney/cancer-genomics...,False,/graph/png/github.com/tmooney/cancer-genomics-...,/graph/svg/github.com/tmooney/cancer-genomics-...,/robundle/github.com/tmooney/cancer-genomics-w...,https://w3id.org/cwl/view/git/233f026ffce24007...,NaN,NaN
2,{'repoUrl': 'https://github.com/ebi-wp/webserv...,1648539494435,5df6b762980b15b0f6389149311b82bdd6dff37d,workflow-fetch-hmmscan.cwl,"{'database': {'type': 'String'}, 'method': {'t...",{'workflow_output': {'type': 'File'}},"{'hmmscan_step': {'run': 'hmmer3_hmmscan.cwl',...",3.1.20210922203925,"digraph workflow {\n graph [\n bgcolor = ""...",https://raw.githubusercontent.com/ebi-wp/webse...,https://w3id.org/cwl/view/git/5df6b762980b15b0...,/graph/xdot/github.com/ebi-wp/webservice-cwl/b...,False,/graph/png/github.com/ebi-wp/webservice-cwl/bl...,/graph/svg/github.com/ebi-wp/webservice-cwl/bl...,/robundle/github.com/ebi-wp/webservice-cwl/blo...,https://w3id.org/cwl/view/git/5df6b762980b15b0...,NaN,NaN
3,{'repoUrl': 'https://github.com/ProteinsWebTea...,1648539077414,56dafa4dab5892c5afa35713563dddc78ec5a00d,rRNA_selection.cwl,"{'indexed_sequences': {'type': 'File [FASTA]',...",{'matching_sequences': {'type': 'File [FASTA]'...,{'extract_coord_lines': {'run': 'extract_coord...,1.0.20180525185854,"digraph workflow {\n graph [\n bgcolor = ""...",https://www.apache.org/licenses/LICENSE-2.0,https://w3id.org/cwl/view/git/56dafa4dab5892c5...,/graph/xdot/github.com/ProteinsWebTeam/ebi-met...,False,/graph/png/github.com/ProteinsWebTeam/ebi-meta...,/graph/svg/github.com/ProteinsWebTeam/ebi-meta...,/robundle/github.com/ProteinsWebTeam/ebi-metag...,https://w3id.org/cwl/view/git/56dafa4dab5892c5...,NaN,NaN
4,{'repoUrl': 'https://github.com/rawgene/cwl.gi...,1648538897532,54adf7fcfbf48c9cc4cfcf384e04eede39b5e5f6,hisat2_htseq_dexseq.cwl,"{'annotation': {'type': 'File'}, 'htseq_prepar...","{'samtools_out': {'type': 'Directory'}, 'hisat...",{'htseq_prepare': {'run': '../tools/htseq_prep...,1.0.20180525185854,"digraph workflow {\n graph [\n bgcolor = ""...",https://raw.githubusercontent.com/rawgene/cwl/...,https://w3id.org/cwl/view/git/54adf7fcfbf48c9c...,/graph/xdot/github.com/rawgene/cwl/blob/master...,False,/graph/png/github.com/rawgene/cwl/blob/master/...,/graph/svg/github.com/rawgene/cwl/blob/master/...,/robundle/github.com/rawgene/cwl/blob/master/w...,https://w3id.org/cwl/view/git/54adf7fcfbf48c9c...,NaN,NaN


In [4]:
# Keep only the DB columns
df = df[workflow_columns.values()]
df.head()

,cwltool_version,doc,docker_link,inputs,label,last_commit,license_link,outputs,retrieved_from,retrieved_on,steps,visualisation_dot,ro_bundle_path
0,1.0.20180525185854,NaN,NaN,{'echo_step_output_location': {'type': 'String'}},simple-workflow.cwl,2f7df467b8e38c1ed0ea3c2280c25c49523da6cf,https://raw.githubusercontent.com/stevencox/cw...,{'wf_output': {'type': 'File'}},{'repoUrl': 'https://github.com/stevencox/cwlt...,1648539658432,"{'echo_step': {'run': 'echo.cwl', 'runType': '...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/stevencox/cwltool/blob/ma...
1,3.0.20201203173111,NaN,NaN,"{'p_value': {'type': 'Float (Optional)'}, 'ref...",Varscan Workflow,233f026ffce240071edda526391be0c03186fce8,https://raw.githubusercontent.com/tmooney/canc...,"{'filtered_vcf': {'type': 'File'}, 'unfiltered...",{'repoUrl': 'https://github.com/tmooney/cancer...,1648539591018,"{'filter': {'label': 'fp_filter workflow', 'ru...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/tmooney/cancer-genomics-w...
2,3.1.20210922203925,NaN,NaN,"{'database': {'type': 'String'}, 'method': {'t...",workflow-fetch-hmmscan.cwl,5df6b762980b15b0f6389149311b82bdd6dff37d,https://raw.githubusercontent.com/ebi-wp/webse...,{'workflow_output': {'type': 'File'}},{'repoUrl': 'https://github.com/ebi-wp/webserv...,1648539494435,"{'hmmscan_step': {'run': 'hmmer3_hmmscan.cwl',...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/ebi-wp/webservice-cwl/blo...
3,1.0.20180525185854,NaN,NaN,"{'indexed_sequences': {'type': 'File [FASTA]',...",rRNA_selection.cwl,56dafa4dab5892c5afa35713563dddc78ec5a00d,https://www.apache.org/licenses/LICENSE-2.0,{'matching_sequences': {'type': 'File [FASTA]'...,{'repoUrl': 'https://github.com/ProteinsWebTea...,1648539077414,{'extract_coord_lines': {'run': 'extract_coord...,"digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/ProteinsWebTeam/ebi-metag...
4,1.0.20180525185854,NaN,NaN,"{'annotation': {'type': 'File'}, 'htseq_prepar...",hisat2_htseq_dexseq.cwl,54adf7fcfbf48c9cc4cfcf384e04eede39b5e5f6,https://raw.githubusercontent.com/rawgene/cwl/...,"{'samtools_out': {'type': 'Directory'}, 'hisat...",{'repoUrl': 'https://github.com/rawgene/cwl.gi...,1648538897532,{'htseq_prepare': {'run': '../tools/htseq_prep...,"digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/rawgene/cwl/blob/master/w...


In [5]:
# Pre-generate the IDs
pd.options.mode.chained_assignment = None  # default='warn'

df['id'] = df.apply(lambda x: str(uuid4()), axis=1)
df = df.set_index('id')
df.head()

,cwltool_version,doc,docker_link,inputs,label,last_commit,license_link,outputs,retrieved_from,retrieved_on,steps,visualisation_dot,ro_bundle_path
id,,,,,,,,,,,,,
25ed0209-e784-4863-b0c4-a797d821b228,1.0.20180525185854,NaN,NaN,{'echo_step_output_location': {'type': 'String'}},simple-workflow.cwl,2f7df467b8e38c1ed0ea3c2280c25c49523da6cf,https://raw.githubusercontent.com/stevencox/cw...,{'wf_output': {'type': 'File'}},{'repoUrl': 'https://github.com/stevencox/cwlt...,1648539658432,"{'echo_step': {'run': 'echo.cwl', 'runType': '...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/stevencox/cwltool/blob/ma...
75037b48-dbd4-49b3-abd3-d387d3472272,3.0.20201203173111,NaN,NaN,"{'p_value': {'type': 'Float (Optional)'}, 'ref...",Varscan Workflow,233f026ffce240071edda526391be0c03186fce8,https://raw.githubusercontent.com/tmooney/canc...,"{'filtered_vcf': {'type': 'File'}, 'unfiltered...",{'repoUrl': 'https://github.com/tmooney/cancer...,1648539591018,"{'filter': {'label': 'fp_filter workflow', 'ru...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/tmooney/cancer-genomics-w...
029195cc-1e1b-48b8-9d7c-89980c8b0214,3.1.20210922203925,NaN,NaN,"{'database': {'type': 'String'}, 'method': {'t...",workflow-fetch-hmmscan.cwl,5df6b762980b15b0f6389149311b82bdd6dff37d,https://raw.githubusercontent.com/ebi-wp/webse...,{'workflow_output': {'type': 'File'}},{'repoUrl': 'https://github.com/ebi-wp/webserv...,1648539494435,"{'hmmscan_step': {'run': 'hmmer3_hmmscan.cwl',...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/ebi-wp/webservice-cwl/blo...
241a5dca-f323-4b91-8922-f115469a2d32,1.0.20180525185854,NaN,NaN,"{'indexed_sequences': {'type': 'File [FASTA]',...",rRNA_selection.cwl,56dafa4dab5892c5afa35713563dddc78ec5a00d,https://www.apache.org/licenses/LICENSE-2.0,{'matching_sequences': {'type': 'File [FASTA]'...,{'repoUrl': 'https://github.com/ProteinsWebTea...,1648539077414,{'extract_coord_lines': {'run': 'extract_coord...,"digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/ProteinsWebTeam/ebi-metag...
6a33ca9e-1a7c-4523-beea-3049ebbc7c08,1.0.20180525185854,NaN,NaN,"{'annotation': {'type': 'File'}, 'htseq_prepar...",hisat2_htseq_dexseq.cwl,54adf7fcfbf48c9cc4cfcf384e04eede39b5e5f6,https://raw.githubusercontent.com/rawgene/cwl/...,"{'samtools_out': {'type': 'Directory'}, 'hisat...",{'repoUrl': 'https://github.com/rawgene/cwl.gi...,1648538897532,{'htseq_prepare': {'run': '../tools/htseq_prep...,"digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/rawgene/cwl/blob/master/w...


In [6]:
%%time

# JSON columns must be output as JSON, so that it's {"id": 1} and not {'id': 1} (plain text in CSV)

json_columns = [
    'retrieved_from',
    'inputs',
    'outputs',
    'steps'
]
for column in json_columns:
    df[column] = df[column].apply(json.dumps)

df.head()

CPU times: user 142 ms, sys: 8.44 ms, total: 150 ms
Wall time: 149 ms


,cwltool_version,doc,docker_link,inputs,label,last_commit,license_link,outputs,retrieved_from,retrieved_on,steps,visualisation_dot,ro_bundle_path
id,,,,,,,,,,,,,
25ed0209-e784-4863-b0c4-a797d821b228,1.0.20180525185854,NaN,NaN,"{""echo_step_output_location"": {""type"": ""String""}}",simple-workflow.cwl,2f7df467b8e38c1ed0ea3c2280c25c49523da6cf,https://raw.githubusercontent.com/stevencox/cw...,"{""wf_output"": {""type"": ""File""}}","{""repoUrl"": ""https://github.com/stevencox/cwlt...",1648539658432,"{""echo_step"": {""run"": ""echo.cwl"", ""runType"": ""...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/stevencox/cwltool/blob/ma...
75037b48-dbd4-49b3-abd3-d387d3472272,3.0.20201203173111,NaN,NaN,"{""p_value"": {""type"": ""Float (Optional)""}, ""ref...",Varscan Workflow,233f026ffce240071edda526391be0c03186fce8,https://raw.githubusercontent.com/tmooney/canc...,"{""filtered_vcf"": {""type"": ""File""}, ""unfiltered...","{""repoUrl"": ""https://github.com/tmooney/cancer...",1648539591018,"{""filter"": {""label"": ""fp_filter workflow"", ""ru...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/tmooney/cancer-genomics-w...
029195cc-1e1b-48b8-9d7c-89980c8b0214,3.1.20210922203925,NaN,NaN,"{""database"": {""type"": ""String""}, ""method"": {""t...",workflow-fetch-hmmscan.cwl,5df6b762980b15b0f6389149311b82bdd6dff37d,https://raw.githubusercontent.com/ebi-wp/webse...,"{""workflow_output"": {""type"": ""File""}}","{""repoUrl"": ""https://github.com/ebi-wp/webserv...",1648539494435,"{""hmmscan_step"": {""run"": ""hmmer3_hmmscan.cwl"",...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/ebi-wp/webservice-cwl/blo...
241a5dca-f323-4b91-8922-f115469a2d32,1.0.20180525185854,NaN,NaN,"{""indexed_sequences"": {""type"": ""File [FASTA]"",...",rRNA_selection.cwl,56dafa4dab5892c5afa35713563dddc78ec5a00d,https://www.apache.org/licenses/LICENSE-2.0,"{""matching_sequences"": {""type"": ""File [FASTA]""...","{""repoUrl"": ""https://github.com/ProteinsWebTea...",1648539077414,"{""extract_coord_lines"": {""run"": ""extract_coord...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/ProteinsWebTeam/ebi-metag...
6a33ca9e-1a7c-4523-beea-3049ebbc7c08,1.0.20180525185854,NaN,NaN,"{""annotation"": {""type"": ""File""}, ""htseq_prepar...",hisat2_htseq_dexseq.cwl,54adf7fcfbf48c9cc4cfcf384e04eede39b5e5f6,https://raw.githubusercontent.com/rawgene/cwl/...,"{""samtools_out"": {""type"": ""Directory""}, ""hisat...","{""repoUrl"": ""https://github.com/rawgene/cwl.gi...",1648538897532,"{""htseq_prepare"": {""run"": ""../tools/htseq_prep...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/rawgene/cwl/blob/master/w...


In [7]:
%%time

df['retrieved_on'] = pd.to_datetime(df['retrieved_on'], unit='ms')
df.head()

CPU times: user 3.52 ms, sys: 0 ns, total: 3.52 ms
Wall time: 2.89 ms


,cwltool_version,doc,docker_link,inputs,label,last_commit,license_link,outputs,retrieved_from,retrieved_on,steps,visualisation_dot,ro_bundle_path
id,,,,,,,,,,,,,
25ed0209-e784-4863-b0c4-a797d821b228,1.0.20180525185854,NaN,NaN,"{""echo_step_output_location"": {""type"": ""String""}}",simple-workflow.cwl,2f7df467b8e38c1ed0ea3c2280c25c49523da6cf,https://raw.githubusercontent.com/stevencox/cw...,"{""wf_output"": {""type"": ""File""}}","{""repoUrl"": ""https://github.com/stevencox/cwlt...",2022-03-29 07:40:58.432,"{""echo_step"": {""run"": ""echo.cwl"", ""runType"": ""...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/stevencox/cwltool/blob/ma...
75037b48-dbd4-49b3-abd3-d387d3472272,3.0.20201203173111,NaN,NaN,"{""p_value"": {""type"": ""Float (Optional)""}, ""ref...",Varscan Workflow,233f026ffce240071edda526391be0c03186fce8,https://raw.githubusercontent.com/tmooney/canc...,"{""filtered_vcf"": {""type"": ""File""}, ""unfiltered...","{""repoUrl"": ""https://github.com/tmooney/cancer...",2022-03-29 07:39:51.018,"{""filter"": {""label"": ""fp_filter workflow"", ""ru...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/tmooney/cancer-genomics-w...
029195cc-1e1b-48b8-9d7c-89980c8b0214,3.1.20210922203925,NaN,NaN,"{""database"": {""type"": ""String""}, ""method"": {""t...",workflow-fetch-hmmscan.cwl,5df6b762980b15b0f6389149311b82bdd6dff37d,https://raw.githubusercontent.com/ebi-wp/webse...,"{""workflow_output"": {""type"": ""File""}}","{""repoUrl"": ""https://github.com/ebi-wp/webserv...",2022-03-29 07:38:14.435,"{""hmmscan_step"": {""run"": ""hmmer3_hmmscan.cwl"",...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/ebi-wp/webservice-cwl/blo...
241a5dca-f323-4b91-8922-f115469a2d32,1.0.20180525185854,NaN,NaN,"{""indexed_sequences"": {""type"": ""File [FASTA]"",...",rRNA_selection.cwl,56dafa4dab5892c5afa35713563dddc78ec5a00d,https://www.apache.org/licenses/LICENSE-2.0,"{""matching_sequences"": {""type"": ""File [FASTA]""...","{""repoUrl"": ""https://github.com/ProteinsWebTea...",2022-03-29 07:31:17.414,"{""extract_coord_lines"": {""run"": ""extract_coord...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/ProteinsWebTeam/ebi-metag...
6a33ca9e-1a7c-4523-beea-3049ebbc7c08,1.0.20180525185854,NaN,NaN,"{""annotation"": {""type"": ""File""}, ""htseq_prepar...",hisat2_htseq_dexseq.cwl,54adf7fcfbf48c9cc4cfcf384e04eede39b5e5f6,https://raw.githubusercontent.com/rawgene/cwl/...,"{""samtools_out"": {""type"": ""Directory""}, ""hisat...","{""repoUrl"": ""https://github.com/rawgene/cwl.gi...",2022-03-29 07:28:17.532,"{""htseq_prepare"": {""run"": ""../tools/htseq_prep...","digraph workflow {\n graph [\n bgcolor = ""...",/robundle/github.com/rawgene/cwl/blob/master/w...


In [8]:
%%time
import csv

# Convert to CSV

df.iloc[0:5].to_csv('cwlviewer_import_me.csv')

CPU times: user 5.55 ms, sys: 0 ns, total: 5.55 ms
Wall time: 4.48 ms
